In [12]:
import os
import json
from pptx import Presentation
from pptx.util import Inches
from pathlib import Path
from pptx.enum.text import PP_ALIGN


# ✅ 画像フォルダの定義 (ローカルテスト用)
BASE_DIR = "pictures"
PRODUCTS_DIR = os.path.join(BASE_DIR, "products-pictures")
PARTS_DIR = os.path.join(BASE_DIR, "parts-pictures")

# ✅ 疑似POSTデータ (フロントエンドから送られる想定)
mock_post_data = json.dumps({
    "product_images": [
        "products-pictures/productA.png",
        "products-pictures/productB.png"
    ]
})

def get_parts_list(product_img_path):
    """製品画像のパスから対応する部品画像リストを取得"""
    product_name = Path(product_img_path).stem  # `productA.jpg` → `productA`
    parts_folder = os.path.join(PARTS_DIR, product_name)

    # ✅ 画像フォーマットリスト（サポートする拡張子を指定）
    valid_extensions = {".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff"}

    part_images = []
    if os.path.exists(parts_folder):
        # ✅ 指定の拡張子を持つファイルのみ取得
        part_images = [str(p) for p in Path(parts_folder).iterdir() if p.suffix.lower() in valid_extensions]

    return product_name, part_images

def add_combined_slide(prs, product_name, product_img, part_imgs):
    """製品と部品を1枚のスライドに収めるように動的に配置"""
    slide = prs.slides.add_slide(prs.slide_layouts[5])  # タイトルなしのスライド
    
    # ✅ タイトルテキストボックスを明示的に作成し、横幅を確保
    title_box = slide.shapes.add_textbox(Inches(1), Inches(0.2), Inches(8), Inches(0.5))
    title_frame = title_box.text_frame
    title_frame.text = product_name
    title_frame.paragraphs[0].alignment = PP_ALIGN.CENTER  # 中央揃え
    title_frame.word_wrap = False  # 改行を防ぐ
    # title_frame.auto_size = True  # 自動サイズ調整を有効化

    # ✅ 製品画像の初期サイズ
    prod_left = Inches(1.0)
    prod_top = Inches(0.8)  
    prod_width = Inches(7.5)  # 初期値
    prod_height = Inches(3.5)  # 初期値

    # ✅ 部品画像の初期サイズ
    part_width = Inches(1.5)
    part_height = Inches(1.5)

    margin_x = Inches(0.3)  # 画像間の水平マージン
    margin_y = Inches(0.3)  # 画像間の垂直マージン

    start_x = Inches(0.5)  # 部品画像の左端
    start_y = Inches(4.0)  # 製品画像の下

    slide_width = Inches(10)  # スライドの幅
    slide_height = Inches(7.5)  # スライドの高さ
    available_height = slide_height - start_y  # 部品を配置できる最大の高さ

    max_cols = int((slide_width - start_x) / (part_width + margin_x))  # 横に並べる最大列数
    max_rows = len(part_imgs) // max_cols + (1 if len(part_imgs) % max_cols else 0)  # 必要な行数

    # ✅ 部品が多すぎてはみ出る場合、製品画像と部品を小さく調整
    while (max_rows * (part_height + margin_y) > available_height) and prod_height > Inches(2.5):
        prod_height -= Inches(0.3)  # 製品画像を少しずつ小さくする
        start_y += Inches(0.2)  # 部品画像の開始位置を上にずらす
        available_height = slide_height - start_y
        part_height = part_width = max(Inches(1.0), part_height - Inches(0.2))  # 部品も縮小
        max_rows = len(part_imgs) // max_cols + (1 if len(part_imgs) % max_cols else 0)  # 再計算

    # ✅ 製品画像を調整後のサイズで配置
    slide.shapes.add_picture(product_img, prod_left, prod_top, prod_width, prod_height)

    # ✅ 部品画像を調整後のサイズで配置
    for i, part_img in enumerate(part_imgs):
        row = i // max_cols
        col = i % max_cols

        left = start_x + (part_width + margin_x) * col
        top = start_y + (part_height + margin_y) * row

        slide.shapes.add_picture(part_img, left, top, part_width, part_height)

def create_pptx(product_image_paths):
    """受け取った製品画像リストを使って PowerPoint を作成"""
    prs = Presentation()

    for product_img_path in product_image_paths:
        full_path = os.path.join(BASE_DIR, product_img_path)
        if not os.path.exists(full_path):
            print(f"⚠️ 製品画像が見つかりません: {full_path}")
            continue

        product_name, part_images = get_parts_list(product_img_path)

        add_combined_slide(prs, product_name, full_path, part_images)

    pptx_filename = "output_slide.pptx"
    prs.save(pptx_filename)
    return pptx_filename

# ✅ 疑似POSTリクエストの受信
def mock_lambda_handler(mock_event):
    """Lambda の挙動を模倣"""
    try:
        # ✅ リクエストボディの解析
        body = json.loads(mock_event)
        product_image_paths = body.get("product_images", [])

        if not product_image_paths:
            return {"statusCode": 400, "body": json.dumps({"error": "製品画像が指定されていません"})}

        # ✅ PowerPoint を作成
        pptx_path = create_pptx(product_image_paths)

        return {
            "statusCode": 200,
            "body": json.dumps({"message": "スライド作成成功", "pptx_file": pptx_path}),
            "headers": {"Content-Type": "application/json"}
        }

    except Exception as e:
        return {"statusCode": 500, "body": json.dumps({"error": str(e)})}

# ✅ 疑似POSTの実行
response = mock_lambda_handler(mock_post_data)

# ✅ 実行結果の確認
print(response)

{'statusCode': 200, 'body': '{"message": "\\u30b9\\u30e9\\u30a4\\u30c9\\u4f5c\\u6210\\u6210\\u529f", "pptx_file": "output_slide.pptx"}', 'headers': {'Content-Type': 'application/json'}}
